In [1]:
import pandas as pd
import settings 
Link = settings.EXTRACTED_DIR # path to extracted files
Link2 = settings.PROCESSED_DIR # path to save modified files

In [6]:
# DATASET-1
dataset1 = pd.read_csv(Link + "\\" + 'Demand_2014_2020.csv')
dataset1.head(5)
dataset1_0 = dataset1[["REGION", "SETTLEMENTDATE", "TOTALDEMAND"]]

#Change columnnames 
dataset1_1 = dataset1_0.rename(columns={'REGION':'STATE', 'SETTLEMENTDATE':'DATE_OBJECT', 'TOTALDEMAND':'DEMAND'})

#Change Statenames
dataset1_1["STATE"].value_counts() # Check State names
dataset1_1["STATE"].replace(["VIC1", "TAS1", "NSW1","SA1", "QLD1"], ["VIC", "TAS", "NSW", "SA", "QLD"], inplace= True)
dataset1_1["STATE"].value_counts() # Check if state names are modified correctly

#Get Date objects - Time, day, date, year
dataset1_1["DATE_OBJECT"].value_counts()
dataset1_1['DATE'] = pd.to_datetime(dataset1_1["DATE_OBJECT"]).dt.date
dataset1_1['YEAR'] = pd.to_datetime(dataset1_1['DATE']).dt.year
dataset1_1['DAY'] = pd.to_datetime(dataset1_1['DATE_OBJECT']).dt.day
dataset1_1['MONTH'] = pd.to_datetime(dataset1_1['DATE']).dt.month
dataset1_1['HOUR'] = pd.to_datetime(dataset1_1['DATE_OBJECT']).dt.hour
dataset1_1 ['MINUTES']= pd.to_datetime(dataset1_1['DATE_OBJECT']).dt.minute

dataset1_1.head(5) # Chcek if all columns are set properly

# Treating missing values of Demand
# No missing values are there for Demand variable in dataset

# Dataset at half  hour level
dataset1_1.drop(['DATE_OBJECT'], axis='columns', inplace = True)
file_name = "Demand_halfhourly.csv"
dataset1_1.to_csv(Link2 + "\\" + file_name, index=False)

# Dataset at hour level
dataset1_1.drop(['MINUTES'], axis='columns', inplace = True)
df1 = dataset1_1.groupby(by=["HOUR","DAY","DATE","MONTH","YEAR","STATE"], as_index=False)["DEMAND"].sum()
file_name = "Demand_hourly.csv"
df1.to_csv(Link2 + "\\" + file_name, index=False)

# Aggregate data for a day, as we have temperature data at day level 
#( We can extract data for temperature at hour level, but to keep analysis simple and data size easy to work on, 
# analysis will be conducted at day level)
dataset1_1.drop(['HOUR'], axis='columns', inplace = True)
df1 = dataset1_1.groupby(by=["DAY","DATE", "MONTH","YEAR","STATE"],as_index=False)["DEMAND"].sum()
file_name = "Demand_daily.csv"
df1.to_csv(Link2 + "\\" + file_name, index=False)

# Aggregate the data at month and State level
dataset1_1.drop(['DAY', 'DATE'], axis='columns', inplace = True)
df1 = dataset1_1.groupby(by=["MONTH","YEAR","STATE"], as_index=False)["DEMAND"].sum()
file_name = "Demand_monthly.csv"
df1.to_csv(Link2 + "\\" + file_name, index=False)

In [7]:
# DATASET-2- PART 1
dataset2 = pd.read_csv(Link + "\\" + 'Temp_State_Daily_Max_2014_2020.csv')
dataset2.head(5)
# Keep Desried columns
dataset2_0 = dataset2[["Year", "Month", "Day", "Maximum temperature (Degree C)", 
                       "File Identifier"]]

# Getting State info from file identifier and dropping the column File identifier
dataset2_1 = dataset2_0.assign(STATE = lambda x: (x['File Identifier'].str)[:3])
dataset2_1.drop(['File Identifier'], axis='columns', inplace = True)
dataset2_1.head(5)

#Change columnnames 
dataset2_2 = dataset2_1.rename(columns={'Year':'YEAR', 'Month':'MONTH', 'Day':'DAY', 
                                        "Maximum temperature (Degree C)":'MAX_TEMP'})

#Change Statenames
dataset2_2["STATE"].value_counts() # Check State names
dataset2_2["STATE"].replace(["SA_", "NT_", "Que", "WA_"], ["SA", "NT", "QLD", "WA"], inplace= True)
dataset2_2["STATE"].value_counts() # Check if state names are modified correctly


# Keeping dataset consist across all sources . We select only these states for which Demand data is avaliable
#["VIC", "TAS", "NSW", "SA", "QLD"]
dataset2_3 = dataset2_2.loc[~dataset2_2["STATE"].isin(["WA", "NT", "ACT"])]
dataset2_3["STATE"].value_counts() # Checking if other states are dropped

#Treating missing values of Max temperature and Min Temperature
# As we have high resolution data, we can interpolate missing values based on time, State
dataset2_3.loc[dataset2_3.MAX_TEMP.isnull(), "MAX_TEMP"] = dataset2_3.groupby(["MONTH","STATE"]).MAX_TEMP.transform('mean')

# Saving modified file daily
file_name = "Temp_daily_Max.csv"
dataset2_3.to_csv(Link2 + "\\" + file_name, index=False)

# # # Making Monthly file
dataset2_3.drop(['DAY'], axis='columns', inplace = True)

file_name = "Temp_monthly_Max.csv"
df = dataset2_3.groupby(by=["MONTH","YEAR","STATE"], as_index=False)["MAX_TEMP"].mean()
df.to_csv(Link2 + "\\" + file_name, index=False)

C:\Users\raman\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [8]:
# DATASET-2- PART 1
dataset2 = pd.read_csv(Link + "\\" + 'Temp_State_Daily_Min_2014_2020.csv')
dataset2.head(5)
# Keep Desried columns
dataset2_0 = dataset2[["Year", "Month", "Day", "Minimum temperature (Degree C)", 
                       "File Identifier"]]

# Getting State info from file identifier and dropping the column File identifier
dataset2_1 = dataset2_0.assign(STATE = lambda x: (x['File Identifier'].str)[:3])
dataset2_1.drop(['File Identifier'], axis='columns', inplace = True)
dataset2_1.head(5)

#Change columnnames 
dataset2_2 = dataset2_1.rename(columns={'Year':'YEAR', 'Month':'MONTH', 'Day':'DAY', 
                                        "Minimum temperature (Degree C)":'MIN_TEMP'})

#Change Statenames
dataset2_2["STATE"].value_counts() # Check State names
dataset2_2["STATE"].replace(["SA_", "NT_", "Que", "WA_"], ["SA", "NT", "QLD", "WA"], inplace= True)
dataset2_2["STATE"].value_counts() # Check if state names are modified correctly


# Keeping dataset consist across all sources . We select only these states for which Demand data is avaliable
#["VIC", "TAS", "NSW", "SA", "QLD"]
dataset2_3 = dataset2_2[~dataset2_2["STATE"].isin(["WA", "NT", "ACT"])]
dataset2_3["STATE"].value_counts() # Checking if other states are dropped

#Treating missing values of Max temperature and Min Temperature
# As we have high resolution data, we can interpolate missing values based on time, State
dataset2_3.loc[dataset2_3.MIN_TEMP.isnull(), "MIN_TEMP"] = dataset2_3.groupby(["MONTH","STATE"]).MIN_TEMP.transform('mean')

# Saving modified file daily
file_name = "Temp_daily_Min.csv"
dataset2_3.to_csv(Link2 + "\\" + file_name, index=False)

# # # Making Monthly file
dataset2_3.drop(['DAY'], axis='columns', inplace = True)

file_name = "Temp_monthly_Min.csv"
df = dataset2_3.groupby(by=["MONTH","YEAR","STATE"], as_index=False)["MIN_TEMP"].mean()
df.to_csv(Link2 + "\\" + file_name, index=False)

In [9]:
# DATASET-3 PART-1
dataset3 = pd.read_excel(Link + "\\" + 'Employment_Unemployment_state_month_2014_2020.xls')

# Getting desired columns using melt function
dataset3_0 = dataset3.melt(id_vars = ['Month'],
             var_name = "State",
             value_name = "EPR")

#Change Statenames
dataset3_1 = dataset3_0.assign(STATE = lambda x:(x['State'].str)[47:])
dataset3_1["STATE"].replace(["South Australia ;", "Western Australia ;", "Queensland ;", "Northern Territory ;", 
                             "Australian Capital Territory ;",  "Victoria ;", "New South Wales ;", "Tasmania ;" ], 
                            ["SA", "WA", "QLD", "NT", "ACT", "VIC", "NSW", "TAS"], inplace= True)
#dataset3_1["STATE"].value_counts() 

# Keeping dataset consist across all sources . We select only these states for which Demand data is avaliable
#["VIC", "TAS", "NSW", "SA", "QLD"]
dataset3_2 = dataset3_1.loc[~dataset3_1["STATE"].isin(["WA", "NT", "ACT"])]
#dataset3_2["STATE"].value_counts() # Checking if other states are dropped

# Changing Date element
dataset3_2['DATE'] = pd.to_datetime(dataset3_2["Month"]).dt.date
dataset3_2['YEAR'] = pd.to_datetime(dataset3_2['Month']).dt.year
dataset3_2['MONTH'] = pd.to_datetime(dataset3_2['Month']).dt.month
dataset3_2.drop(['Month', 'State'], axis='columns', inplace = True)

# Save the modified file
file_name = "EmploymentStatistics_Monthly.csv"
dataset3_2.to_csv(Link2 + "\\" + file_name, index=False)

<ipython-input-9-63073b3064b4>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset3_2['DATE'] = pd.to_datetime(dataset3_2["Month"]).dt.date
<ipython-input-9-63073b3064b4>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset3_2['YEAR'] = pd.to_datetime(dataset3_2['Month']).dt.year
<ipython-input-9-63073b3064b4>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [10]:
# DATASET-3 PART-2
dataset3 = pd.read_excel(Link + "\\" + 'Employment_Unemployment_state_month_2014_2020.xls', sheet_name = "Unemployment Rate")
# Getting desired columns using melt function
dataset3_0 = dataset3.melt(id_vars = ['Month'],
             var_name = "State",
             value_name = "Unemploy_Rate")

#Change Statenames
dataset3_1 = dataset3_0.assign(STATE = lambda x:(x['State'].str)[34:])
dataset3_1["STATE"].replace(["South Australia ;", "Western Australia ;", "Queensland ;", "Northern Territory ;", 
                              "Australian Capital Territory ;",  "Victoria ;", "New South Wales ;", "Tasmania ;" ], 
                             ["SA", "WA", "QLD", "NT", "ACT", "VIC", "NSW", "TAS"], inplace= True)
dataset3_1["STATE"].value_counts() 

# Keeping dataset consist across all sources . We select only these states for which Demand data is avaliable
# ["VIC", "TAS", "NSW", "SA", "QLD"]
dataset3_2 = dataset3_1.loc[~dataset3_1["STATE"].isin(["WA", "NT", "ACT"])]
#dataset3_2["STATE"].value_counts() # Checking if other states are dropped

# # Changing Date element
dataset3_2['DATE'] = pd.to_datetime(dataset3_2["Month"]).dt.date
dataset3_2['YEAR'] = pd.to_datetime(dataset3_2['Month']).dt.year
dataset3_2['MONTH'] = pd.to_datetime(dataset3_2['Month']).dt.month
dataset3_2.drop(['Month', 'State'], axis='columns', inplace = True)
# dataset3_2.head(3)

# # Save the modified file
file_name = "UnemploymentStatistics_Monthly.csv"
dataset3_2.to_csv(Link2 + "\\" + file_name, index=False)


<ipython-input-10-c63eae46983d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset3_2['DATE'] = pd.to_datetime(dataset3_2["Month"]).dt.date
<ipython-input-10-c63eae46983d>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset3_2['YEAR'] = pd.to_datetime(dataset3_2['Month']).dt.year
<ipython-input-10-c63eae46983d>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [11]:
#DATASET -3 PART 3
dataset3 = pd.read_excel(Link + "\\" + 'Employment_Unemployment_state_month_2014_2020.xls', sheet_name = "Population")
# Getting desired columns using melt function
dataset3_0 = dataset3.melt(id_vars = ['Month'],
             var_name = "State",
             value_name = "Population")

#Change Statenames
dataset3_1 = dataset3_0.assign(STATE = lambda x:(x['State'].str)[22:])
dataset3_1["STATE"].replace(["South Australia ;", "Western Australia ;", "Queensland ;", "Northern Territory ;", 
                              "Australian Capital Territory ;",  "Victoria ;", "New South Wales ;", "Tasmania ;" ], 
                            ["SA", "WA", "QLD", "NT", "ACT", "VIC", "NSW", "TAS"], inplace= True)

# Keeping dataset consist across all sources . We select only these states for which Demand data is avaliable
# ["VIC", "TAS", "NSW", "SA", "QLD"]
dataset3_2 = dataset3_1.loc[~dataset3_1["STATE"].isin(["WA", "NT", "ACT"])]
#dataset3_2["STATE"].value_counts() # Checking if other states are dropped

# # # Changing Date element
dataset3_2['DATE'] = pd.to_datetime(dataset3_2.loc[:, 'Month']).dt.date
dataset3_2['YEAR'] = pd.to_datetime(dataset3_2.loc[:, 'Month']).dt.year
dataset3_2['MONTH'] = pd.to_datetime(dataset3_2.loc[:, 'Month']).dt.month
dataset3_2.drop(['Month', 'State'], axis='columns', inplace = True)

# # # Save the modified file
file_name = "Population_Monthly.csv"
dataset3_2.to_csv(Link2 + "\\" + file_name, index=False)

<ipython-input-11-0685c8acd9da>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset3_2['DATE'] = pd.to_datetime(dataset3_2.loc[:, 'Month']).dt.date
<ipython-input-11-0685c8acd9da>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset3_2['YEAR'] = pd.to_datetime(dataset3_2.loc[:, 'Month']).dt.year
<ipython-input-11-0685c8acd9da>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [12]:
# DATASET-4
dataset4 = pd.read_csv(Link + "\\" + 'Australian_public_holidays_2014_2020.csv')

# Keeping desired columns
dataset4_1 = dataset4[['Date', 'Applicable To', 'Jurisdiction']]

# Breaking Applicable to : i.e. NSW|SA|WA into different states
dataset4_1[["S1", "S2", "S3", "S4", "S5", "S6", "S7"]] = pd.DataFrame(dataset4_1['Applicable To'].str.split("|", expand = True))

#Breaking NAT ( National Holiday) into different states
dataset4_1.loc[dataset4_1['Applicable To'] == 'NAT', ['S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7']] = ['NSW', 'SA', 'WA', 'QLD', 'TAS', 'NT', 'ACT']

# for year 2019 & 2020, state is recorded under column jurisdiction
dataset4_1['S8'] = None 
dataset4_1.loc[dataset4_1['Jurisdiction'] != None, 'S8']= dataset4_1['Jurisdiction'].str.upper()
dataset4_1.loc[dataset4_1['Applicable To'] == 'NAT', 'S8']= 'VIC'

# Getting Desired columns 
dataset4_2 = dataset4_1[['Date', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8']]

# Getting desired columns using melt function
dataset4_3 = dataset4_2.melt(id_vars = ['Date'],
             var_name = "state",
             value_name = "STATE")

# Deleting NaN Values and other states

dataset4_4 = dataset4_3.loc[~dataset4_3["STATE"].isin(["WA", "NT", "ACT", None])]
dataset4_5 = dataset4_4[['Date', 'STATE']]

# Adding Holiday Flag
dataset4_5['Holiday_Flag'] = 1

# Date objects
dataset4_5['DATE'] = pd.to_datetime(dataset4_5["Date"], format='%Y%m%d').dt.date
dataset4_5['DAY'] = pd.to_datetime(dataset4_5["DATE"]).dt.day
dataset4_5['YEAR'] = pd.to_datetime(dataset4_5['DATE']).dt.year
dataset4_5['MONTH'] = pd.to_datetime(dataset4_5['DATE']).dt.month

dataset4_5.drop(['Date'], axis='columns', inplace = True)

# Saving File
# Save the modified file
file_name = "Hoilidays.csv"
dataset4_5.to_csv(Link2 + "\\" + file_name, index=False)